## 一个情感分类的例子，输入是电影评论，输出是；label：0 或1。0代表消极评论；1代表积极评论。

<img src='imgs/1.png' >

In [6]:
#torchtext:用来处理文本的一个包

In [4]:
# !pip install torchtext

In [3]:
# !pip install spacy

In [11]:
# !python -m spacy download en_core_web_sm

## 1，读取数据

In [1]:
import torch
#data：用来生成field
from torchtext.legacy import data

SEED = 455

#在需要生成随机数的实验中，确保每次运行.py文件时，
#生成的随机数都是固定的，这样每次实验结果显示也就一致了。
#设定random seeds使实验可以复现。

torch.manual_seed(SEED)

In [2]:
# torch.manual_seed(1)
# torch.rand(1,2)

In [3]:
torch.backends.cudnn.deterministic = True   # 固定cuda的seed

In [5]:
# Field 包含一些文本处理的通用参数的设置，同时还包含一个词典对象，可以把文本数据表示成数字类型，
# 进而可以把文本表示成需要的tensor类型
# tokenize：The function used to tokenize strings using this field into
#           sequential examples. 
    
# tokenize='spacy'，这表示我们会用spaCy tokenizer来tokenize英文句子。
# 如果我们不特别声明tokenize这个参数，那么默认的分词方法是使用空格。

# tokenizer_language:tells torchtext which spaCy model to use.

# Field :主要包含以下数据预处理的配置信息，比如指定分词方法，是否转成小写，
#     起始字符，结束字符，补全字符以及词典等等    
    
TEXT = data.Field(tokenize = 'spacy',
                 tokenizer_language = 'en_core_web_sm')

In [6]:
help(data.Field)

Help on class Field in module torchtext.legacy.data.field:

class Field(RawField)
 |  Field(sequential=True, use_vocab=True, init_token=None, eos_token=None, fix_length=None, dtype=torch.int64, preprocessing=None, postprocessing=None, lower=False, tokenize=None, tokenizer_language='en', include_lengths=False, batch_first=False, pad_token='<pad>', unk_token='<unk>', pad_first=False, truncate_first=False, stop_words=None, is_target=False)
 |  
 |  Defines a datatype together with instructions for converting to Tensor.
 |  
 |  Field class models common text processing datatypes that can be represented
 |  by tensors.  It holds a Vocab object that defines the set of possible values
 |  for elements of the field and their corresponding numerical representations.
 |  The Field object also holds other parameters relating to how a datatype
 |  should be numericalized, such as a tokenization method and the kind of
 |  Tensor that should be produced.
 |  
 |  If a Field is shared between two co

In [7]:
LABEL = data.LabelField(dtype = torch.float)   # 必须指明float

In [8]:
# 生成数据集
from torchtext.legacy import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL,root='.data')

downloading aclImdb_v1.tar.gz


100%|█████████████████████████████████████████████████████████████████████████████| 84.1M/84.1M [00:26<00:00, 3.19MB/s]


In [10]:
help(datasets.IMDB.splits)

Help on method splits in module torchtext.legacy.datasets.imdb:

splits(text_field, label_field, root='.data', train='train', test='test', **kwargs) method of builtins.type instance
    Create dataset objects for splits of the IMDB dataset.
    
    Args:
        text_field: The field that will be used for the sentence.
        label_field: The field that will be used for label data.
        root: Root dataset storage directory. Default is '.data'.
        train: The directory that contains the training examples
        test: The directory that contains the test examples
        Remaining keyword arguments: Passed to the splits method of
            Dataset.



In [14]:
# The IMDb dataset consists of 50,000 movie reviews, 
# each marked as being a positive or negative review.

In [14]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [15]:
train_data.examples[0]

In [16]:
# vars() 函数返回对象object的属性和属性值的字典对象。
print(vars(train_data.examples[0]))

{'text': ['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', '"', 'Teachers', '"', '.', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', 'High', "'s", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"', 'Teachers', '"', '.', 'The', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', "'", 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'I', 'knew', 'and', 'their', 'students', '.', 'When', 'I', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 'I', 'immediately', 'recalled', '.........', 'at', '..........', 'High', '.', 'A', 'classic', 'l

In [17]:
help(train_data.split)

Help on method split in module torchtext.legacy.data.dataset:

split(split_ratio=0.7, stratified=False, strata_field='label', random_state=None) method of torchtext.legacy.datasets.imdb.IMDB instance
    Create train-test(-valid?) splits from the instance's examples.
    
    Arguments:
        split_ratio (float or List of floats): a number [0, 1] denoting the amount
            of data to be used for the training split (rest is used for test),
            or a list of numbers denoting the relative sizes of train, test and valid
            splits respectively. If the relative size for valid is missing, only the
            train-test split is returned. Default is 0.7 (for the train set).
        stratified (bool): whether the sampling should be stratified.
            Default is False.
        strata_field (str): name of the examples Field stratified over.
            Default is 'label' for the conventional label field.
        random_state (tuple): the random seed used for shuffling

In [18]:
import random
train_data, valid_data = train_data.split(random_state = random.seed(SEED)) 

In [19]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


## 2，建立词典
<img src="imgs/2.png" >

In [20]:
#我们建立一个25000的词典，不在词典的用<unk>表示
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

#### 这里面有2个特殊字符\<unk>和\<pad>

In [21]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [22]:
# stoi:s to i
TEXT.vocab.stoi['<unk']

0

In [23]:
TEXT.vocab.itos[2]

'the'

In [74]:
# 对于一个batch里的数据，不够长度的用<pad>填充

In [24]:
help(TEXT.build_vocab)

Help on method build_vocab in module torchtext.legacy.data.field:

build_vocab(*args, **kwargs) method of torchtext.legacy.data.field.Field instance
    Construct the Vocab object for this field from one or more datasets.
    
    Arguments:
        Positional arguments: Dataset objects or other iterable data
            sources from which to construct the Vocab object that
            represents the set of possible values for this field. If
            a Dataset object is provided, all columns corresponding
            to this field are used; individual columns can also be
            provided directly.
        Remaining keyword arguments: Passed to the constructor of Vocab.



<img src="imgs/3.png">

In [71]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202979), (',', 192741), ('.', 165709), ('a', 109773), ('and', 109607), ('of', 100946), ('to', 93686), ('is', 76282), ('in', 61365), ('I', 53604), ('it', 53386), ('that', 49188), ('"', 43962), ("'s", 43580), ('this', 42171), ('-', 37189), ('/><br', 35640), ('was', 34659), ('as', 30637), ('with', 30064)]


In [72]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [73]:
print(LABEL.vocab.stoi)

defaultdict(None, {'pos': 0, 'neg': 1})


In [83]:
#把数据转换成迭代器的形式，data.BucketIterator.splits

In [74]:
BATCH_SIZE = 64

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

#### 获取一个例子查看

In [75]:
a_batch_train_examples = next(iter(train_iterator))

In [28]:
# sizet为[sen len, batch size]

In [29]:
a_batch_train_examples.text[0]

tensor([   25,    66,  1216, 12879,  3288,    55, 14068,   607,   643,  1448,
           11,   149,   149,   787,    11,   112,   607,  3665,  1379,  5386,
        17210,    11,  9086,    66,  2537,    25,    11,    11,  1512,   370,
          149,   156,   544,    66,   149,    14,    66,  1433,   462,  4182,
           11,   908,   806,    11,    66,  3146,   806,  8654,   462,   272,
           14,   402,    11,     2,   149,   170,    66,    11,  6716,  1552,
           66,   472,    11,  1267], device='cuda:0')

In [76]:
a_batch_train_examples.label[0]

tensor(0., device='cuda:0')

## 3，建立模型

<img src="imgs/4.png">

In [32]:
#### 1,Each batch, text is a tensor of size [sentence length, batch size]
#### 2,embedded is a tensor of size [sentence length, batch size, embedding dim].
#### 3, output of size [sentence length, batch size, hidden dim] 
#### 4,hidden of size [1, batch size, hidden dim]. 

In [77]:
import torch.nn as nn

In [78]:
# [0,2,0.3,0.5]
# [0.6]

In [79]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sen len, batch size]
        # 在这里，我们不需要指明one-hot的维度，pytorch会自动把索引转换为one-hot向量。
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [35]:
# The embedding dimension is the size of the dense word vectors. 
# This is usually around 50-250 dimensions, but depends on the size of the vocabulary.

# The hidden dimension is the size of the hidden states. 
# This is usually around 100-500 dimensions, 
# but also depends on factors such as on the vocabulary size, 
# the size of the dense vectors and the complexity of the task.

In [80]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [81]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


##  4,Train the Model

In [83]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [82]:
criterion = nn.BCEWithLogitsLoss()#包含sigmoid和交叉熵损失

In [84]:
model = model.to(device)
criterion = criterion.to(device)

In [41]:
### 定义正确率函数

In [88]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [87]:
# torch.round(torch.tensor(0.6))

In [42]:
# model.train() is used to put the model in "training mode", 
# which turns on dropout and batch normalization. 
# Although we aren't using them in this model, 
# it's good practice to include it.

In [43]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)#size: [batch_size,1]
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [120]:
# model.eval() puts the model in "evaluation mode", 
# this turns off dropout and batch normalization. Again, 
# we are not using them in this model, but it is good practice to include them.

# No gradients are calculated on PyTorch operations inside the with no_grad() block. 
# This causes less memory to be used and speeds up computation.

In [45]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()   # 关闭dropout，正则化
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [46]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [49]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
#     else:
#         break
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 32s
	Train Loss: 0.694 | Train Acc: 49.88%
	 Val. Loss: 0.697 |  Val. Acc: 50.55%
Epoch: 02 | Epoch Time: 0m 31s
	Train Loss: 0.693 | Train Acc: 49.99%
	 Val. Loss: 0.697 |  Val. Acc: 49.67%
Epoch: 03 | Epoch Time: 0m 31s
	Train Loss: 0.693 | Train Acc: 50.35%
	 Val. Loss: 0.697 |  Val. Acc: 49.32%
Epoch: 04 | Epoch Time: 0m 33s
	Train Loss: 0.693 | Train Acc: 50.21%
	 Val. Loss: 0.697 |  Val. Acc: 49.47%
Epoch: 05 | Epoch Time: 0m 32s
	Train Loss: 0.693 | Train Acc: 50.24%
	 Val. Loss: 0.697 |  Val. Acc: 49.54%
